In [1]:
from __future__ import print_function
from matplotlib import pyplot as plt
# get_ipython().magic(u'matplotlib auto')
import tensorflow as tf
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()
# import torch
from tensorflow import keras
import statistics

In [2]:
# pip install -q -U tensorflow-addons

In [3]:
# import tensorflow_addons as tfa
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Activation
from tensorflow import reshape
from keras.utils import np_utils
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv1D,Conv2D, MaxPooling1D,AveragePooling1D
from tensorflow.keras.layers import SeparableConv1D, Bidirectional
from tensorflow.keras.layers import LocallyConnected2D,LocallyConnected1D
from tensorflow.keras.layers import ZeroPadding2D,ZeroPadding1D, MaxPooling2D, Bidirectional
from tensorflow.keras.regularizers import l2,l1
from tensorflow.keras.layers import BatchNormalization,Add,concatenate
from tensorflow.keras.callbacks import CSVLogger,LearningRateScheduler
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

In [4]:
from scipy.io import loadmat
import numpy as np
import pandas as pd
import seaborn as sns
from numpy import mean
from numpy import std
# import coremltools
# from torch import nn, optim
# import torch.nn.functional as F
import scipy.io
from scipy import stats
#from IPython.display import display, HTML
from tensorflow.keras import optimizers
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import TimeDistributed
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn import preprocessing

In [5]:
N_FEATURES = 8

In [6]:
column_names = ['C'+str(j) for j in range(1, N_FEATURES+1)]
lst = ['Class_label']
column_names = column_names+lst
print(column_names)

['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'Class_label']


In [7]:
def read_data_Train(file_path_Train):
    df_Train = pd.read_csv(file_path_Train,header=None,names=column_names)
    # Last column has a ";" character which must be removed ...
    df_Train['Class_label'].replace(regex=True,inplace=True,to_replace=r';',value=r'')
    # ... and then this column must be transformed to float explicitly
    df_Train['Class_label'] = df_Train['Class_label'].apply(convert_to_float)
    # This is very important otherwise the model will not fit and loss
    # will show up as NAN
    df_Train.dropna(axis=0, how='any', inplace=True)
    return df_Train
def convert_to_float(x):
    try:
        return np.float(x)
    except:
        return np.nan

In [8]:
def show_basic_dataframe_info(dataframe):
    # Shape and how many rows and columns
    print('Number of columns in the dataframe: %i' % (dataframe.shape[1]))
    print('Number of rows in the dataframe: %i\n' % (dataframe.shape[0]))

In [9]:
def read_data_Test(file_path_Test):
    df_Test = pd.read_csv(file_path_Test,header=None,names=column_names)
    # Last column has a ";" character which must be removed ...
    df_Test['Class_label'].replace(regex=True,inplace=True,to_replace=r';',value=r'')
    # ... and then this column must be transformed to float explicitly
    df_Test['Class_label'] = df_Test['Class_label'].apply(convert_to_float)
    # This is very important otherwise the model will not fit and loss
    # will show up as NAN
    df_Test.dropna(axis=0, how='any', inplace=True)
    return df_Test

In [10]:
 pd.options.display.float_format = "{:,.5f}".format

In [11]:
# 15 function to segment data into trial lengths (trial length =513 samples in this dataset)
def create_segments_and_labels(df, time_steps,step,n_features, label_name):
    segments = []
    labels = []
    for i in range(0, len(df) - time_steps, step):
      for j in range(1, n_features+1):
        L = ('C'+str(j)) 
        segments.append(df[str(L)].values[i: i + time_steps])
      label = stats.mode(df[label_name][i: i + time_steps])[0][0]
      labels.append(label)
    reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, time_steps, n_features)
    labels = np.asarray(labels)
    return reshaped_segments, labels

In [12]:
# import os
path='/media/naveen/nav/mat_codes/BioPatRec/prep_LP_wave/Total_process_TT/'

In [13]:
#  for i in range(num_res_net_blocks_64):
#     x = res_net_block_64(x, 64, 3)

In [14]:
# file_path_train=path+'Train_data'+'.'+'csv'
file_path_train=path+'Biopatrec_26c_17s_LP_wave_train'+'.'+'csv'
print(file_path_train)

/media/naveen/nav/mat_codes/BioPatRec/prep_LP_wave/Total_process_TT/Biopatrec_26c_17s_LP_wave_train.csv


In [15]:
file_path_test=path+'Biopatrec_26c_17s_LP_wave_test'+'.'+'csv'
# file_path_test=path+'Test_data'+'.'+'csv'
print(file_path_test)

/media/naveen/nav/mat_codes/BioPatRec/prep_LP_wave/Total_process_TT/Biopatrec_26c_17s_LP_wave_test.csv


In [16]:
df_Train=read_data_Train(file_path_train)
show_basic_dataframe_info(df_Train)
df_Train.head(5)

Number of columns in the dataframe: 9
Number of rows in the dataframe: 5304000



,C1,C2,C3,C4,C5,C6,C7,C8,Class_label
0,0.03450,0.04517,0.03171,0.03034,0.02866,0.02575,0.02943,0.03246,1.00000
1,0.03450,0.04517,0.03171,0.03034,0.02866,0.02575,0.02942,0.03245,1.00000
2,0.03450,0.04517,0.03172,0.03033,0.02866,0.02575,0.02942,0.03245,1.00000
3,0.03450,0.04517,0.03172,0.03033,0.02866,0.02575,0.02942,0.03245,1.00000
4,0.03450,0.04516,0.03173,0.03033,0.02866,0.02575,0.02942,0.03245,1.00000


In [17]:
df_Test=read_data_Test(file_path_test)
show_basic_dataframe_info(df_Test)
df_Test.head(5)

Number of columns in the dataframe: 9
Number of rows in the dataframe: 2652000



,C1,C2,C3,C4,C5,C6,C7,C8,Class_label
0,0.03051,0.04162,0.02142,0.03620,0.03281,0.02426,0.02407,0.03174,1.00000
1,0.03037,0.04162,0.02144,0.03620,0.03281,0.02426,0.02407,0.03174,1.00000
2,0.03020,0.04162,0.02145,0.03619,0.03280,0.02426,0.02407,0.03174,1.00000
3,0.03000,0.04162,0.02146,0.03618,0.03280,0.02426,0.02408,0.03174,1.00000
4,0.02982,0.04162,0.02147,0.03617,0.03279,0.02426,0.02408,0.03174,1.00000


In [18]:
pd.options.display.float_format = "{:,.5f}".format
df_Train.head(5)

,C1,C2,C3,C4,C5,C6,C7,C8,Class_label
0,0.03450,0.04517,0.03171,0.03034,0.02866,0.02575,0.02943,0.03246,1.00000
1,0.03450,0.04517,0.03171,0.03034,0.02866,0.02575,0.02942,0.03245,1.00000
2,0.03450,0.04517,0.03172,0.03033,0.02866,0.02575,0.02942,0.03245,1.00000
3,0.03450,0.04517,0.03172,0.03033,0.02866,0.02575,0.02942,0.03245,1.00000
4,0.03450,0.04516,0.03173,0.03033,0.02866,0.02575,0.02942,0.03245,1.00000


In [19]:
scaler = preprocessing.StandardScaler().fit(df_Train.iloc[:,0:N_FEATURES])

In [20]:
df_Train.iloc[:,0:N_FEATURES]=scaler.transform(df_Train.iloc[:,0:N_FEATURES])

In [21]:
pd.options.display.float_format = "{:,.5f}".format
df_Train.head(5)

,C1,C2,C3,C4,C5,C6,C7,C8,Class_label
0,-0.40862,-0.38335,-0.31710,-0.19573,-0.20851,-0.25914,-0.23657,-0.25710,1.00000
1,-0.40862,-0.38336,-0.31707,-0.19576,-0.20853,-0.25915,-0.23658,-0.25713,1.00000
2,-0.40861,-0.38336,-0.31704,-0.19579,-0.20853,-0.25916,-0.23659,-0.25717,1.00000
3,-0.40861,-0.38337,-0.31700,-0.19582,-0.20853,-0.25918,-0.23660,-0.25719,1.00000
4,-0.40860,-0.38338,-0.31697,-0.19585,-0.20854,-0.25919,-0.23661,-0.25723,1.00000


In [22]:
pd.options.display.float_format = "{:,.5f}".format
df_Test.head(5)

,C1,C2,C3,C4,C5,C6,C7,C8,Class_label
0,0.03051,0.04162,0.02142,0.03620,0.03281,0.02426,0.02407,0.03174,1.00000
1,0.03037,0.04162,0.02144,0.03620,0.03281,0.02426,0.02407,0.03174,1.00000
2,0.03020,0.04162,0.02145,0.03619,0.03280,0.02426,0.02407,0.03174,1.00000
3,0.03000,0.04162,0.02146,0.03618,0.03280,0.02426,0.02408,0.03174,1.00000
4,0.02982,0.04162,0.02147,0.03617,0.03279,0.02426,0.02408,0.03174,1.00000


In [23]:
df_Test.iloc[:,0:N_FEATURES]=scaler.transform(df_Test.iloc[:,0:N_FEATURES])

In [24]:
pd.options.display.float_format = "{:,.5f}".format
df_Test.head(5)

,C1,C2,C3,C4,C5,C6,C7,C8,Class_label
0,-0.44073,-0.40073,-0.40371,-0.10845,-0.15168,-0.27363,-0.28653,-0.26682,1.00000
1,-0.44187,-0.40073,-0.40361,-0.10855,-0.15175,-0.27362,-0.28651,-0.26683,1.00000
2,-0.44322,-0.40072,-0.40351,-0.10867,-0.15182,-0.27362,-0.28649,-0.26686,1.00000
3,-0.44482,-0.40072,-0.40341,-0.10878,-0.15189,-0.27361,-0.28647,-0.26687,1.00000
4,-0.44625,-0.40072,-0.40331,-0.10889,-0.15196,-0.27361,-0.28644,-0.26689,1.00000


In [25]:
LABEL = 'ActivityEncoded'
# Transform the labels from String to Integer via LabelEncoder
le = preprocessing.LabelEncoder()
# Add a new column to the existing DataFrame with the encoded values
df_Train[LABEL] = le.fit_transform(df_Train['Class_label'].values.ravel())
# df_Valid[LABEL] = le.fit_transform(df_Valid['Class_label'].values.ravel())
df_Test[LABEL] = le.fit_transform(df_Test['Class_label'].values.ravel())

In [26]:
# The number of steps within one time segment
TIME_PERIODS = 300
# The steps to take from one segment to the next; if this value is equal to
# TIME_PERIODS, then there is no overlap between the segments
STEP_DISTANCE = 300

In [27]:
x_train, y_train = create_segments_and_labels(df_Train,TIME_PERIODS,STEP_DISTANCE,N_FEATURES,LABEL)
print('x_train shape: ', x_train.shape)
# print(x_train)
print(x_train.shape[0], 'training samples')
print('y_train shape: ', y_train.shape)
# Set input & output dimensions
num_time_periods, num_sensors = x_train.shape[1], x_train.shape[2]
print('num_time_periods',num_time_periods)
print('num_sensors',num_sensors)
num_classes = le.classes_.size
print('class_list',list(le.classes_))
# input_shape = (num_time_periods,num_sensors)
# print(input_shape)
input_shape = (num_time_periods,num_sensors)
#x_train = x_train.reshape(x_train.shape[0], input_shape)
print('x_train shape:', x_train[0].shape)
print('input_shape:', input_shape)
x_train = x_train.astype('float32')
# x_train = [torch.tensor(arr, dtype=torch.float32) for arr in x_train]
# y_train = y_train.astype('float32')
# print(y_train)
y_train_hot = np_utils.to_categorical(y_train, num_classes)
print(y_train_hot)
# y_train_hot= [torch.tensor(arr, dtype=torch.uint8) for arr in y_train_hot]
print('New y_train shape: ', y_train_hot.shape)

x_train shape:  (17679, 300, 8)
17679 training samples
y_train shape:  (17679,)
num_time_periods 300
num_sensors 8
class_list [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0]
x_train shape: (300, 8)
input_shape: (300, 8)
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]
New y_train shape:  (17679, 26)


In [28]:
x_test, y_test = create_segments_and_labels(df_Test,TIME_PERIODS,STEP_DISTANCE,N_FEATURES,LABEL)
print('x_test shape: ', x_test.shape)
# print(x_train)
print(x_test.shape[0], 'testing samples')
print('y_test shape: ', y_test.shape)
# Set input_shape / reshape for Keras
#x_test = x_test.reshape(x_test.shape[0], input_shape)
x_test = x_test.astype('float32')
y_test = y_test.astype('float32')
y_test_hot = np_utils.to_categorical(y_test, num_classes)

x_test shape:  (8839, 300, 8)
8839 testing samples
y_test shape:  (8839,)


In [29]:
# n_steps, n_length = 20, 25
# n_steps, n_length= 10, 50
# n_steps, n_length= 16, 32
n_length =  300
n_depth=8
n_channel=8
x_train = x_train.reshape(x_train.shape[0], n_length,n_depth)
print('x_train shape: ', x_train.shape)
# x_valid = x_valid.reshape(x_valid.shape[0], n_steps, n_length, n_depth)
# print('x_valid shape: ', x_valid.shape)
x_test = x_test.reshape(x_test.shape[0],  n_length,n_depth)
print('x_test shape: ', x_test.shape)
n_outputs = y_train_hot.shape[1]
print('n_outputs',n_outputs)

x_train shape:  (17679, 300, 8)
x_test shape:  (8839, 300, 8)
n_outputs 26


In [30]:
def conv_block(input_data,n_channel):
    inputs=input_data
    print(inputs.shape)
    l_y=[]
    for i in range(0,n_channel):
        ip=inputs[:,:,i]
        print('ip_shape',ip.shape)
#         ip1=tf.convert_to_tensor(ip, dtype=tf.float32)
        ip=tf.reshape(ip,(tf.shape(ip)[0],300,1))
        print('ip_shape',ip.shape)
        x = Conv1D(64, kernel_size=3,kernel_initializer="he_normal",kernel_regularizer=l1(1e-04), \
               padding='same',input_shape=ip.shape)(ip)
        print('conv1_o',x.shape)
        x = BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(x)
        x = Activation('relu')(x)
        x= Conv1D(filters=64, kernel_size=3,padding="same", kernel_regularizer=l2(5e-04),kernel_initializer="he_normal",strides=1)(x)
        x= BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(x)
        x= Activation('relu')(x)
        x= LocallyConnected1D(64, kernel_size=1, kernel_regularizer=l2(5e-04),kernel_initializer="he_normal",strides=1,padding='valid')(x)
        x= BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(x)
        x= Activation('relu')(x)
        x= LocallyConnected1D(64, kernel_size=1, kernel_regularizer=l2(5e-04),kernel_initializer="he_normal",strides=1,padding='valid')(x)
        x= BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(x)
        x= Activation('relu')(x)
        y= Dropout(0.5)(x)
        l_y.append(y)
    y_stack=tf.stack(l_y, axis=2)   
    return y_stack

In [31]:
def generate_model():
    inputs = Input(shape=(n_length,n_depth),name="main")
    p=conv_block(inputs,8)
    print(p.shape)
    print(type(p))
#     print(y.shape)
    y=Flatten()(p)
    print(y.shape)
    y= Dense(512, kernel_initializer="he_normal")(y)
    y= BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(y)
    y= Activation('relu')(y)
    y= Dropout(0.5)(y)
    y= Dense(512, kernel_initializer="he_normal")(y)
    y= BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(y)
    y= Activation('relu')(y)
    y= Dropout(0.5)(y)
    y= Dense(128, kernel_initializer="he_normal")(y)
    y= BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(y)
    y= Activation('relu')(y)
    outputs=Dense(n_outputs, activation='softmax')(y)
    print(outputs.shape)
#     wei_mscnn_model = Model(inputs,sub_inputs], outputs)
    print('inputs_shape',inputs.shape)
    final_model = Model(inputs,outputs,name="wei_mscnn_model")
    return final_model        

In [32]:
if __name__ == "__main__":
    model = generate_model()

(None, 300, 8)
ip_shape (None, 300)
ip_shape (None, 300, 1)
conv1_o (None, 300, 64)
ip_shape (None, 300)
ip_shape (None, 300, 1)
conv1_o (None, 300, 64)
ip_shape (None, 300)
ip_shape (None, 300, 1)
conv1_o (None, 300, 64)
ip_shape (None, 300)
ip_shape (None, 300, 1)
conv1_o (None, 300, 64)
ip_shape (None, 300)
ip_shape (None, 300, 1)
conv1_o (None, 300, 64)
ip_shape (None, 300)
ip_shape (None, 300, 1)
conv1_o (None, 300, 64)
ip_shape (None, 300)
ip_shape (None, 300, 1)
conv1_o (None, 300, 64)
ip_shape (None, 300)
ip_shape (None, 300, 1)
conv1_o (None, 300, 64)
(None, 300, 8, 64)
<class 'tensorflow.python.framework.ops.Tensor'>
(None, 153600)
(None, 26)
inputs_shape (None, 300, 8)


In [33]:
verbose, epochs, batch_size = 0, 200, 16

In [34]:
def step_decay(epoch):
   initial_lrate = 1e-3
   drop = 0.1
   epochs_drop = 50.0
   lrate = initial_lrate * tf.math.pow(drop,  
           tf.math.floor((1+epoch)/epochs_drop))
   return lrate

In [35]:
lrate = tf.keras.callbacks.LearningRateScheduler(step_decay)
test_acc=[]

In [36]:
adam=optimizers.Adam(lr=1e-3, beta_1=0.9, beta_2=0.999, amsgrad=False)
# sgd=tfa.optimizers.SGDW(weight_decay=0.0001,learning_rate=0.1, momentum=0.9, nesterov=False, name='SGDW')
sgd=tf.optimizers.SGD(learning_rate=1e-3, momentum=0.9, nesterov=False, name='SGD')
checkpoint_filepath = '/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/wei_BPR_20X10/checkpoint.hdf5'
# model.load_weights(checkpoint_filepath) 
checkpoint_callback = ModelCheckpoint(filepath=checkpoint_filepath,verbose=1, monitor='val_accuracy',save_weights_only=True,save_best_only=True)
early = EarlyStopping(monitor='val_loss', min_delta=0, patience=100, verbose=0, mode='auto', baseline=None, restore_best_weights=True)
model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])
print(model.summary())
#     tf.keras.utils.plot_model(model, to_file='/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/olsson/CNN20X10/Model1.png',show_shapes=True,show_layer_names=True,dpi=96)

Model: "wei_mscnn_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main (InputLayer)               [(None, 300, 8)]     0                                            
__________________________________________________________________________________________________
tf_op_layer_strided_slice (Tens [(None, 300)]        0           main[0][0]                       
__________________________________________________________________________________________________
tf_op_layer_strided_slice_2 (Te [(None, 300)]        0           main[0][0]                       
__________________________________________________________________________________________________
tf_op_layer_strided_slice_4 (Te [(None, 300)]        0           main[0][0]                       
____________________________________________________________________________________

In [37]:
csv_logger = CSVLogger('/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/wei_BPR_20X10/CNN_nina_20X10.csv', append=True, separator=';')
history = model.fit(x_train, y_train_hot, epochs=epochs, batch_size=batch_size, callbacks=[csv_logger,checkpoint_callback,lrate,early],validation_data=(x_test, y_test_hot), verbose=1)

Epoch 1/200
1105/1105 [==============================] - ETA: 0s - loss: 3.2663 - accuracy: 0.1958
Epoch 00001: val_accuracy improved from -inf to 0.30682, saving model to /media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/wei_BPR_20X10/checkpoint.hdf5
1105/1105 [==============================] - 196s 177ms/step - loss: 3.2663 - accuracy: 0.1958 - val_loss: 2.4988 - val_accuracy: 0.3068 - lr: 0.0010
Epoch 2/200
1105/1105 [==============================] - ETA: 0s - loss: 2.6366 - accuracy: 0.2501
Epoch 00002: val_accuracy improved from 0.30682 to 0.33533, saving model to /media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/wei_BPR_20X10/checkpoint.hdf5
1105/1105 [==============================] - 155s 140ms/step - loss: 2.6366 - accuracy: 0.2501 - val_loss: 2.3010 - val_accuracy: 0.3353 - lr: 0.0010
Epoch 3/200
1105/1105 [==============================] - ETA: 0s - loss: 2.5215 - accuracy: 0.2817
Epoch 00003: val_accuracy improved from 0.33533 to 0.37086, saving model t

1105/1105 [==============================] - 147s 133ms/step - loss: 1.8831 - accuracy: 0.4617 - val_loss: 1.6026 - val_accuracy: 0.5641 - lr: 0.0010
Epoch 23/200
1105/1105 [==============================] - ETA: 0s - loss: 1.8540 - accuracy: 0.4734
Epoch 00023: val_accuracy did not improve from 0.56409
1105/1105 [==============================] - 144s 131ms/step - loss: 1.8540 - accuracy: 0.4734 - val_loss: 1.6228 - val_accuracy: 0.5453 - lr: 0.0010
Epoch 24/200
1105/1105 [==============================] - ETA: 0s - loss: 1.8482 - accuracy: 0.4803
Epoch 00024: val_accuracy improved from 0.56409 to 0.57416, saving model to /media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/wei_BPR_20X10/checkpoint.hdf5
1105/1105 [==============================] - 147s 133ms/step - loss: 1.8482 - accuracy: 0.4803 - val_loss: 1.5903 - val_accuracy: 0.5742 - lr: 0.0010
Epoch 25/200
1105/1105 [==============================] - ETA: 0s - loss: 1.8222 - accuracy: 0.4849
Epoch 00025: val_accuracy did

Epoch 71/200
1105/1105 [==============================] - ETA: 0s - loss: 1.1905 - accuracy: 0.6491
Epoch 00071: val_accuracy did not improve from 0.62428
1105/1105 [==============================] - 150s 136ms/step - loss: 1.1905 - accuracy: 0.6491 - val_loss: 1.2796 - val_accuracy: 0.6162 - lr: 1.0000e-04
Epoch 72/200
1105/1105 [==============================] - ETA: 0s - loss: 1.2084 - accuracy: 0.6434
Epoch 00072: val_accuracy did not improve from 0.62428
1105/1105 [==============================] - 149s 135ms/step - loss: 1.2084 - accuracy: 0.6434 - val_loss: 1.2874 - val_accuracy: 0.6134 - lr: 1.0000e-04
Epoch 73/200
1105/1105 [==============================] - ETA: 0s - loss: 1.1961 - accuracy: 0.6495
Epoch 00073: val_accuracy did not improve from 0.62428
1105/1105 [==============================] - 149s 135ms/step - loss: 1.1961 - accuracy: 0.6495 - val_loss: 1.2896 - val_accuracy: 0.6132 - lr: 1.0000e-04
Epoch 74/200
1105/1105 [==============================] - ETA: 0s - loss:

Epoch 98/200
1105/1105 [==============================] - ETA: 0s - loss: 1.0893 - accuracy: 0.6781
Epoch 00098: val_accuracy did not improve from 0.62428
1105/1105 [==============================] - 143s 129ms/step - loss: 1.0893 - accuracy: 0.6781 - val_loss: 1.2785 - val_accuracy: 0.6098 - lr: 1.0000e-04
Epoch 99/200
1105/1105 [==============================] - ETA: 0s - loss: 1.0805 - accuracy: 0.6848
Epoch 00099: val_accuracy did not improve from 0.62428
1105/1105 [==============================] - 143s 129ms/step - loss: 1.0805 - accuracy: 0.6848 - val_loss: 1.3160 - val_accuracy: 0.6012 - lr: 1.0000e-04
Epoch 100/200
1105/1105 [==============================] - ETA: 0s - loss: 1.0690 - accuracy: 0.6897
Epoch 00100: val_accuracy did not improve from 0.62428
1105/1105 [==============================] - 143s 130ms/step - loss: 1.0690 - accuracy: 0.6897 - val_loss: 1.2808 - val_accuracy: 0.6108 - lr: 1.0000e-05
Epoch 101/200
1105/1105 [==============================] - ETA: 0s - los

Epoch 125/200
1105/1105 [==============================] - ETA: 0s - loss: 1.0465 - accuracy: 0.6886
Epoch 00125: val_accuracy did not improve from 0.62428
1105/1105 [==============================] - 143s 130ms/step - loss: 1.0465 - accuracy: 0.6886 - val_loss: 1.2816 - val_accuracy: 0.6083 - lr: 1.0000e-05
Epoch 126/200
1105/1105 [==============================] - ETA: 0s - loss: 1.0269 - accuracy: 0.6988
Epoch 00126: val_accuracy did not improve from 0.62428
1105/1105 [==============================] - 143s 130ms/step - loss: 1.0269 - accuracy: 0.6988 - val_loss: 1.2836 - val_accuracy: 0.6123 - lr: 1.0000e-05
Epoch 127/200
1105/1105 [==============================] - ETA: 0s - loss: 1.0330 - accuracy: 0.6943
Epoch 00127: val_accuracy did not improve from 0.62428
1105/1105 [==============================] - 143s 130ms/step - loss: 1.0330 - accuracy: 0.6943 - val_loss: 1.2979 - val_accuracy: 0.6049 - lr: 1.0000e-05
Epoch 128/200
1105/1105 [==============================] - ETA: 0s - l

Epoch 152/200
1105/1105 [==============================] - ETA: 0s - loss: 1.0299 - accuracy: 0.6983
Epoch 00152: val_accuracy did not improve from 0.62428
1105/1105 [==============================] - 143s 130ms/step - loss: 1.0299 - accuracy: 0.6983 - val_loss: 1.2870 - val_accuracy: 0.6074 - lr: 1.0000e-06
Epoch 153/200
1105/1105 [==============================] - ETA: 0s - loss: 1.0206 - accuracy: 0.7000
Epoch 00153: val_accuracy did not improve from 0.62428
1105/1105 [==============================] - 143s 129ms/step - loss: 1.0206 - accuracy: 0.7000 - val_loss: 1.2918 - val_accuracy: 0.6080 - lr: 1.0000e-06
Epoch 154/200
1105/1105 [==============================] - ETA: 0s - loss: 1.0256 - accuracy: 0.7009
Epoch 00154: val_accuracy did not improve from 0.62428
1105/1105 [==============================] - 144s 130ms/step - loss: 1.0256 - accuracy: 0.7009 - val_loss: 1.2922 - val_accuracy: 0.6049 - lr: 1.0000e-06
Epoch 155/200
1105/1105 [==============================] - ETA: 0s - l

Epoch 179/200
1105/1105 [==============================] - ETA: 0s - loss: 1.0247 - accuracy: 0.6993
Epoch 00179: val_accuracy did not improve from 0.62428
1105/1105 [==============================] - 143s 130ms/step - loss: 1.0247 - accuracy: 0.6993 - val_loss: 1.2804 - val_accuracy: 0.6096 - lr: 1.0000e-06
Epoch 180/200
1105/1105 [==============================] - ETA: 0s - loss: 1.0251 - accuracy: 0.6946
Epoch 00180: val_accuracy did not improve from 0.62428
1105/1105 [==============================] - 143s 129ms/step - loss: 1.0251 - accuracy: 0.6946 - val_loss: 1.2917 - val_accuracy: 0.6026 - lr: 1.0000e-06
Epoch 181/200
1105/1105 [==============================] - ETA: 0s - loss: 1.0237 - accuracy: 0.7021
Epoch 00181: val_accuracy did not improve from 0.62428
1105/1105 [==============================] - 143s 130ms/step - loss: 1.0237 - accuracy: 0.7021 - val_loss: 1.2833 - val_accuracy: 0.6106 - lr: 1.0000e-06


In [38]:
best_index = history.history['val_accuracy'].index(max(history.history['val_accuracy']))
print('epoch_number',best_index+1)
print('train accuracy and validation accuracy', history.history['accuracy'][best_index], history.history['val_accuracy'][best_index])

epoch_number 53
train accuracy and validation accuracy 0.6114599108695984 0.6242787837982178


In [39]:
model.load_weights(checkpoint_filepath) 
_, testaccuracy = model.evaluate(x_test, y_test_hot, batch_size=batch_size, verbose=1)
print('test_accuracy',testaccuracy)
test_acc.append(testaccuracy)
print(test_acc)

553/553 [==============================] - 19s 34ms/step - loss: 1.3320 - accuracy: 0.6243
test_accuracy 0.6242787837982178
[0.6242787837982178]


In [40]:
statistics.mean(test_acc)

0.6242787837982178